# Imitation Minimal Viable Example

This notebook tests an imitation system using other demonstrations as pseudo live views.

1. This uses optical flow for tracking correspondence.
2. It just tracks one center keypoint.


## Load Recorded Sequences

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
from typing import List, Dict

import ipywidgets
import numpy as np
import open3d as o3d
from tqdm import tqdm
import matplotlib.pyplot as plt

import casino
#from DITTO.data import Hands23Dataset, get_all_runs
#from DITTO.config import BASE_RECORDING_PATH, TIME_STEPS
# from DITTO.tracking_3D import Step3DMethod


# Activate the interactive stuff
# https://github.com/microsoft/vscode-jupyter/wiki/Using-%25matplotlib-widget-instead-of-%25matplotlib-notebook,tk,etc
import ipympl
%matplotlib widget
# %matplotlib inline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Notebook settings - DON'T USE
PLOT_2D_TRAJECTORIES = False
PLOT_3D_TRAJECTORIES = True

session_id = "coke_tray"
all_runs = get_all_runs(only_keywords=[str(session_id)])

loaders: List[Hands23Dataset] = []
for ep in tqdm(all_runs, desc="Loading"):
    loaders.append(Hands23Dataset(session_id / ep, lazy_loading=True))

# loaders: List[Hands23Dataset] = [
#    Hands23Dataset(session_id / "000"),
#    Hands23Dataset(session_id / "001")
# Hands23Dataset(session_id / "004") # Demo --> Live pose estimation is not optimal
# Hands23Dataset(session_id / "001") # Demo --> Live pose estimation is not optimal
# ]

print(f"Total runs {len(loaders)}")

NameError: name 'get_all_runs' is not defined

In [11]:
# MV
from src.my_loader import DetectAndSegment
from pathlib import Path

#   1 data/droid_raw/1.0.1/success/2023-03-02/Thu_Mar__2_15_03_35_2023
#   2 data/droid_raw/1.0.1/success/2023-03-02/Thu_Mar__2_16_51_12_2023
#   3 data/droid_raw/1.0.1/success/2023-03-02/Thu_Mar__2_18_13_38_2023 +
#   4 data/droid_raw/1.0.1/success/2023-03-03/Fri_Mar__3_15_30_17_2023
#   5 data/droid_raw/1.0.1/success/2023-03-06/Mon_Mar__6_16_02_37_2023 +
#   6 data/droid_raw/1.0.1/success/2023-03-06/Mon_Mar__6_17_06_52_2023 X too dark
#   7 data/droid_raw/1.0.1/success/2023-03-07/Tue_Mar__7_15_36_10_2023
#   8 data/droid_raw/1.0.1/success/2023-03-07/Tue_Mar__7_16_20_41_2023
#   9 data/droid_raw/1.0.1/success/2023-03-07/Tue_Mar__7_17_59_19_2023
#  10 data/droid_raw/1.0.1/success/2023-03-08/Wed_Mar__8_13_29_47_2023
#  11 data/droid_raw/1.0.1/success/2023-03-08/Wed_Mar__8_14_45_19_2023
#  12 data/droid_raw/1.0.1/success/2023-03-08/Wed_Mar__8_16_45_10_2023 X
#  13 data/droid_raw/1.0.1/success/2023-03-08/Wed_Mar__8_19_32_03_2023
#  14 data/droid_raw/1.0.1/success/2023-03-08/Wed_Mar__8_19_51_18_2023
#  15 data/droid_raw/1.0.1/success/2023-03-09/Thu_Mar__9_18_14_34_2023 X
#  16 data/droid_raw/1.0.1/success/2023-03-09/Thu_Mar__9_18_47_35_2023 +
#  17 data/droid_raw/1.0.1/success/2023-03-09/Thu_Mar__9_19_48_15_2023
#  18 data/droid_raw/1.0.1/success/2023-04-07/Fri_Apr__7_13_32_40_2023 X
#  19 data/droid_raw/1.0.1/success/2023-04-12/Wed_Apr_12_14:13:32_2023 OK
#  20 data/droid_raw/1.0.1/success/2023-04-12/Wed_Apr_12_15:09:51_2023

# src/raw.py --visualize  --scene data/droid_raw/1.0.1/success/2023-04-07/Fri_Apr__7_13_32_40_2023
scene =                          "data/droid_raw/1.0.1/success/2023-03-08/Wed_Mar__8_16_45_10_2023"
loader = DetectAndSegment(Path(scene))
loader.read_trajectory()
loaders: List = [loader]


episode sec 10
frames 152
fps 14.20
frame 1
gripper closed False
frame 2
gripper closed False
frame 3
gripper closed False
frame 4
gripper closed False
frame 5
gripper closed False
frame 6
gripper closed False
frame 7
gripper closed False
frame 8
gripper closed False
frame 9
gripper closed False
frame 10
gripper closed False
frame 11
gripper closed False
frame 12
gripper closed False
frame 13
gripper closed False
frame 14
gripper closed False
frame 15
gripper closed False
frame 16
gripper closed False
frame 17
gripper closed False
frame 18
gripper closed False
frame 19
gripper closed False
frame 20
gripper closed False
frame 21
gripper closed False
frame 22
gripper closed False
frame 23
gripper closed False
frame 24
gripper closed False
frame 25
gripper closed False
frame 26
gripper closed False
frame 27
gripper closed False
frame 28
gripper closed False
frame 29
gripper closed False
frame 30
gripper closed False
frame 31
gripper closed False
frame 32
gripper closed False
frame 33
grip

In [12]:
loader.get_start_stop()

(0, 68)

In [13]:
len(loader.rgb)

70

## Initialize Object Trajectories

In [14]:
from DITTO.trajectory import Trajectory

num_frames = -1 # TIME_STEPS  # number of frames through which we compute flow
trajectories: Dict[int, Trajectory] = {}
for demonstration_index in tqdm(range(len(loaders))):
    trajectories[demonstration_index] = Trajectory.from_hands23(loaders[demonstration_index], n_frames=num_frames)

# We could pre compute trajectories with .trajectory_2D and .trajectory_3D

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.72it/s]


In [15]:
trajectory = trajectories[0].trajectory_2D

/home/argusm/lang/RAFT/core/raft.py:99: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.args.mixed_precision):
/home/argusm/lang/RAFT/core/raft.py:110: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.args.mixed_precision):
/home/argusm/lang/RAFT/core/raft.py:127: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.args.mixed_precision):


In [16]:
# we need n points, not n - 1
start, stop = loader.get_start_stop()
n = stop - start + 1
full_trajectory = np.zeros((n, 1, 2))
full_trajectory[1:n] = trajectory
full_trajectory[0] = trajectory[0]
trajectory = full_trajectory

In [17]:
with open("data/trajectory.npy", "wb") as f:
    np.save(f, trajectory)

In [18]:
trajectory

array([[[ 467.,  857.]],

       [[ 467.,  857.]],

       [[ 467.,  857.]],

       [[ 467.,  857.]],

       [[ 467.,  859.]],

       [[ 467.,  859.]],

       [[ 467.,  859.]],

       [[ 467.,  859.]],

       [[ 467.,  859.]],

       [[ 466.,  860.]],

       [[ 465.,  862.]],

       [[ 460.,  868.]],

       [[ 453.,  875.]],

       [[ 443.,  885.]],

       [[ 431.,  897.]],

       [[ 415.,  912.]],

       [[ 405.,  922.]],

       [[ 396.,  930.]],

       [[ 387.,  937.]],

       [[ 382.,  939.]],

       [[ 379.,  940.]],

       [[ 377.,  939.]],

       [[ 375.,  940.]],

       [[ 375.,  941.]],

       [[ 375.,  945.]],

       [[ 376.,  951.]],

       [[ 377.,  957.]],

       [[ 380.,  963.]],

       [[ 384.,  972.]],

       [[ 388.,  979.]],

       [[ 392.,  988.]],

       [[ 394.,  993.]],

       [[ 396.,  996.]],

       [[ 396.,  999.]],

       [[ 395., 1000.]],

       [[ 393., 1000.]],

       [[ 392., 1000.]],

       [[ 392., 1000.]],

       [[ 39

## Show Recordings

Using an interactive viewer.

In [ ]:
from DITTO.notebook_utils import plot_preds
from DITTO.vis_helpers import overlay_mask_edge


def annotate_initial_frame(loader, traj, ax, image, frame_index):
    ax.scatter(*traj.keypoints_2D[0, ::-1], marker="x", c="yellow")
    ax.scatter(*traj.goal_keypoints_2D[0, ::-1], marker="x", c="r")
    if frame_index != 0:
        return image
    start_frame, stop_frame = loader.get_start_stop()
    
    mask = loader.get_object_mask(start_frame)[:, :, 0]
    image = overlay_mask_edge(image, mask, color=(255, 255, 0))
    mask = traj.goal_mask
    image = overlay_mask_edge(image, mask, color=(255, 0, 0))
    return image

plt.close('all')
fig, ax = plt.subplots(1, figsize=(8, 6))
fig.suptitle("Demonstration Frames")
ax.set_axis_off()
image_h = ax.imshow(loaders[demo_index].get_rgb(0))

def update(demo_index, frame_index):
    demo_len = loaders[demo_index].get_len()
    if frame_index >= demo_len:
        print(f"invalid frame index: {frame_index}, demo length: {demo_len}")
        frame_index = demo_len - 1
    image = loaders[demo_index].get_rgb(frame_index)
    mask = loaders[demo_index].get_object_mask(frame_index)[:, :, 0]
    image = overlay_mask_edge(image, mask)
    # plot hands & object bbox preds
    plot_preds(loaders[demo_index], frame_index, ax)
    traj = trajectories[demo_index]
    image = annotate_initial_frame(loaders[demo_index], traj, ax, image, frame_index)
    
    image_h.set_data(image)
    fig.canvas.draw_idle()
    fig.canvas.flush_events()

max_frames_i = max([len(loader) for loader in loaders]) - 1
slider_w = ipywidgets.widgets.IntSlider(
    min=0, max=len(loaders) - 1, step=1, value=0, layout=ipywidgets.Layout(width="70%"))
slider_i = ipywidgets.widgets.IntSlider(
    min=0, max=max_frames_i, step=1, value=0, layout=ipywidgets.Layout(width="70%"))
ipywidgets.interact(update, demo_index=slider_w, frame_index=slider_i)

## Show 3D Traj.

Find the full 3D pose, with rotations, by looking at sequential frames of the demonstration video, and fitting transformations.

This uses RANSAC, however the parameters for this still need to be tuned.

The pointcloud poses look quite good, but there is quite a bit of variance in the pose measurements.

In [43]:
# from DITTO.tracking_3D import Step3DMethod

# trajectories[demo_index]._step_function_3D = Step3DMethod.loftr_ransac
# trajectories[demo_index].reset_cached()

In [52]:
demo_index = 0
live_indices = set(range(len(loaders)))
live_indices.remove(demo_index)


In [49]:
# Plot relative demo trajectory
o3d_objects = []
print(loaders[demo_index].recording_path)

for idx, transform in enumerate(trajectories[demo_index].trajectory_3D):
    abs_point_from_2_5_D = trajectories[demo_index].lifted_2D_trajectory[
        idx, 0, :  # Again, this is all keypoints
    ]

    print(
        f"Point Error [{idx:03d}]: {np.linalg.norm(transform[:3, 3] - abs_point_from_2_5_D) * 1000:.02f} [mm]"
    )

    o3d_objects.append(
        casino.visualization.get_o3d_coordinate_frame(scale=0.1, transform=transform)
    )
    o3d_objects.append(
        casino.visualization.get_o3d_coordinate_frame(
            scale=0.05, position=abs_point_from_2_5_D
        )
    )

pcd = loaders[demo_index].get_pointcloud_o3d(0)

o3d.visualization.draw_geometries(
    [
        pcd,
        *o3d_objects,
    ]
)

/home/heppert/datasets/video_imitation/recordings_24_01_05/tennisball_cup/003


100%|██████████| 10/10 [00:13<00:00,  1.38s/it]


Point Error [000]: 0.00 [mm]
Point Error [001]: 0.78 [mm]
Point Error [002]: 1.19 [mm]
Point Error [003]: 1.14 [mm]
Point Error [004]: 1.36 [mm]
Point Error [005]: 0.98 [mm]
Point Error [006]: 0.79 [mm]
Point Error [007]: 1.38 [mm]
Point Error [008]: 2.87 [mm]
Point Error [009]: 2.37 [mm]
Point Error [010]: 4.47 [mm]


In [ ]:
# Compare the 3D trajectory to the lifted one.
trajectories[demo_index].trajectory_3D

trajectory_3d = trajectories[demo_index].trajectory_3D[..., :3, 3]
trajectory_lifted = trajectories[demo_index].lifted_2D_trajectory[:, 0,  :]

l2_error = np.linalg.norm(trajectory_3d - trajectory_lifted, axis=-1).mean()
print(f"Mean Point Error: {l2_error*1000:.02f} [mm] \t <- 3D vs lifted")

In [ ]:
pc_start, pc_rgb_start = casino.pointcloud.get_pc(
    trajectories[demo_index].depth_start,
    trajectories[demo_index].intrinsics,
    rgb=trajectories[demo_index].rgb_start,
)

pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pc_start))
pcd.colors = o3d.utility.Vector3dVector(pc_rgb_start / 255.0)
o3d.visualization.draw_geometries(
    [
        pcd,
        casino.visualization.get_o3d_coordinate_frame(
            scale=0.1, transform=trajectories[demo_index].hand_pose_3D
        ),
    ]
)

# Warp Demo Traj. to Live View

## Warp 3D with Start

In [ ]:
# Plot relative demo trajectory
for live_index in live_indices:
    o3d_objects = []

    # This could come from anywhere but for now we just use another trajecory
    rgb_live = trajectories[live_index].rgb_start
    xyz_live = casino.pointcloud.get_xyz(
        trajectories[live_index].depth_start, trajectories[live_index].intrinsics
    )

    # this takes a pre-computed trajectory, and demo mask of the initial frame, and warps it the live frame.
    warped_3D_trajectory = trajectories[demo_index].warp_3D_onto_live_frame(
        rgb_live, xyz_live, debug_vis=True
    )

    # Assuming we roughly moved the same, we can visualize the original trajectory
    live_3D_trajectory = trajectories[live_index].trajectory_3D

    for warped_transform, live_transform in zip(
        warped_3D_trajectory, live_3D_trajectory
    ):  # relative transformation between frames
        o3d_objects.append(
            casino.visualization.get_o3d_coordinate_frame(
                scale=0.1, transform=warped_transform
            )
        )

        # o3d_objects.append(
        #     casino.visualization.get_o3d_coordinate_frame(
        #         scale=0.05, transform=live_transform
        #     )
        # )

    # Get scene point cloud
    pc_start, pc_rgb_start = casino.pointcloud.get_pc(
        trajectories[live_index].depth_start,
        trajectories[live_index].intrinsics,
        rgb=trajectories[live_index].rgb_start,
    )

    pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pc_start))
    pcd.colors = o3d.utility.Vector3dVector(pc_rgb_start / 255.0)
    o3d.visualization.draw_geometries(
        [
            pcd,
            *o3d_objects,
        ]
    )

    break

## Warp 3D with Goal

In [ ]:
# Plot relative demo trajectory
for live_index in live_indices:
    # Hard example for coke_tray, when demo_index = 0
    live_index = 5

    o3d_objects = []

    # This could come from anywhere but for now we just use another trajecory
    rgb_live = trajectories[live_index].rgb_start
    xyz_live = casino.pointcloud.get_xyz(
        trajectories[live_index].depth_start, trajectories[live_index].intrinsics
    )

    # this takes a pre-computed trajectory, and demo mask of the initial frame, and warps it the live frame.
    warped_3D_trajectory = trajectories[demo_index].warp_3D_onto_live_frame(
        rgb_live, xyz_live, use_goal_mask=True, debug_vis=True
    )

    # Assuming we roughly moved the same, we can visualize the original trajectory
    live_3D_trajectory = trajectories[live_index].trajectory_3D

    for warped_transform, live_transform in zip(
        warped_3D_trajectory, live_3D_trajectory
    ):  # relative transformation between frames
        o3d_objects.append(
            casino.visualization.get_o3d_coordinate_frame(
                scale=0.1, transform=warped_transform
            )
        )

        # o3d_objects.append(
        #     casino.visualization.get_o3d_coordinate_frame(
        #         scale=0.05, transform=live_transform
        #     )
        # )

    # Get scene point cloud
    pc_start, pc_rgb_start = casino.pointcloud.get_pc(
        trajectories[live_index].depth_start,
        trajectories[live_index].intrinsics,
        rgb=trajectories[live_index].rgb_start,
    )

    pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pc_start))
    pcd.colors = o3d.utility.Vector3dVector(pc_rgb_start / 255.0)
    o3d.visualization.draw_geometries(
        [
            pcd,
            *o3d_objects,
        ]
    )

    break

# Trajectory Mixing

Mixing can only be done, if there is a define goal position.

## Trajectory Mixing in 3D

In [ ]:
from DITTO.mixing import mix_3D

PLOT_3D_TRAJECTORIES = True
from flow_control.utils_coords import matrix_to_pos_orn, pos_orn_to_matrix

from scipy.spatial.transform import Rotation as R

def eval_trajectory_3d(trajectory, estimate, label=""):
    assert trajectory.ndim == 3
    assert trajectory.shape[1] == 4
    assert trajectory.shape[2] == 4
    diff_orn = np.mean((R.from_matrix(trajectory[:,:3,:3]).inv() * R.from_matrix(estimate[:,:3,:3])).magnitude())
    diff_mm = np.mean(np.linalg.norm(trajectory[:, :3, 3] - estimate[:, :3, 3], axis=1))
    print(f"{label}: {diff_mm:.3f}, {diff_orn:.3f} in 3D")
    
    
# Plot relative demo trajectory
for live_index in live_indices:
    # Hard example when coke_tray is selected and demo_index = 0
    live_index = 5 

    # This could come from anywhere but for now we just use another trajecory
    rgb_live = trajectories[live_index].rgb_start
    xyz_live = casino.pointcloud.get_xyz(
        trajectories[live_index].depth_start, trajectories[live_index].intrinsics
    )

    # this takes a pre-computed trajectory, and demo mask of the initial frame, and warps it the live frame.
    object_warped_3D_trajectory = trajectories[demo_index].warp_3D_onto_live_frame(
        rgb_live, xyz_live, use_goal_mask=False
    )
    goal_warped_3D_trajectory = trajectories[demo_index].warp_3D_onto_live_frame(
        rgb_live, xyz_live, use_goal_mask=True
    )

    warped_3D_trajectory = mix_3D(
        object_warped_3D_trajectory, goal_warped_3D_trajectory
    )

    live_3D_trajectory = trajectories[live_index].trajectory_3D

    # Quantatative Evaluation
    eval_trajectory_3d(live_3D_trajectory, object_warped_3D_trajectory, "obj")
    eval_trajectory_3d(live_3D_trajectory, goal_warped_3D_trajectory, "goal")
    eval_trajectory_3d(live_3D_trajectory, warped_3D_trajectory, "mix")

    # visualize
    if PLOT_3D_TRAJECTORIES:
        o3d_objects = []
        
        for warped_transform, live_transform in zip(
            warped_3D_trajectory, live_3D_trajectory
        ):  # relative transformation between frames
            o3d_objects.append(
                casino.visualization.get_o3d_coordinate_frame(
                    scale=0.1, transform=warped_transform
                )
            )

            # o3d_objects.append(
            #     casino.visualization.get_o3d_coordinate_frame(
            #         scale=0.05, transform=live_transform
            #     )
            # )

        # Get scene point cloud
        pc_start, pc_rgb_start = casino.pointcloud.get_pc(
            trajectories[live_index].depth_start,
            trajectories[live_index].intrinsics,
            rgb=trajectories[live_index].rgb_start,
        )

        pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pc_start))
        pcd.colors = o3d.utility.Vector3dVector(pc_rgb_start / 255.0)
        o3d.visualization.draw_geometries(
            [
                pcd,
                *o3d_objects,
            ]
        )
    break

# Warp the Hand Position

In [ ]:
# Plot relative demo trajectory
for live_index in live_indices:
    o3d_objects = []

    # This could come from anywhere but for now we just use another trajecory
    rgb_live = trajectories[live_index].rgb_start
    xyz_live = casino.pointcloud.get_xyz(
        trajectories[live_index].depth_start, trajectories[live_index].intrinsics
    )

    # This warpes the hand
    warped_3D_hand_pose = trajectories[demo_index].warp_3D_hand_onto_live_frame(
        rgb_live, xyz_live
    )

    # Get scene point cloud
    pc_start, pc_rgb_start = casino.pointcloud.get_pc(
        trajectories[live_index].depth_start,
        trajectories[live_index].intrinsics,
        rgb=trajectories[live_index].rgb_start,
    )

    pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pc_start))
    pcd.colors = o3d.utility.Vector3dVector(pc_rgb_start / 255.0)
    o3d.visualization.draw_geometries(
        [
            pcd,
            casino.visualization.get_o3d_coordinate_frame(
                scale=0.1, transform=warped_3D_hand_pose
            ),
        ]
    )